#### MetaData - Prepared manually by taking the Fitness -https://royalsocietypublishing.org/doi/10.1098/rspb.2015.2292 & the allele count data and sequencing data - https://github.com/benjaminhgood/LTEE-metagenomic.

##### import libary

In [ ]:
import sys
sys.path.append("../../..")
# sys.path.append("../../../data/")

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import mannwhitneyu, fisher_exact
import utility, importlib
from scipy.stats import spearmanr
import pingouin as pg
from scipy.stats import linregress
import warnings
import kaos
warnings.filterwarnings('ignore', category=pd.errors.PerformanceWarning)
pd.set_option('mode.chained_assignment', None)

In [ ]:
import kaos
kmer_length =10
file_path_Ecoli = "../../../data/GCF_000017985.1_ASM1798v1_genomic.fna"
seq_fasta = kaos.read_fasta(file_path_Ecoli)

##### Read gain sheet data

In [4]:
df_Gain_calculated = pd.read_excel("../../../data/MetaData_ecoli_final.xlsx", sheet_name= "Gain score")
df_Gain_calculated.head()

,Position,Gene,Allele,Ref_allele,Alt_allele,Annotation,label,accumulated_gain
0,63,intergenic,A->C,A,C,noncoding,p6,-10.385914
1,201,thrL,T->G,T,G,missense,p6,-1.750693
2,241,thrL,A->C,A,C,missense,m1,-0.911836
3,309,thrA,T->G,T,G,noncoding,m1,4.009052
4,322,thrA,A->G,A,G,noncoding,p3,1.532477


In [5]:
df_Gain_calculated[['Position', 'Gene', 'Allele', 'Ref_allele', 'Alt_allele']].drop_duplicates()

,Position,Gene,Allele,Ref_allele,Alt_allele
0,63,intergenic,A->C,A,C
1,201,thrL,T->G,T,G
2,241,thrL,A->C,A,C
3,309,thrA,T->G,T,G
4,322,thrA,A->G,A,G
...,...,...,...,...,...
36916,4629441,lasT,C->T,C,T
36917,4629541,lasT,C->T,C,T
36918,4629573,lasT,T->C,T,C
36919,4629600,lasT,A->G,A,G


In [6]:
df_Gain_calculated[df_Gain_calculated.duplicated(subset=['Position', 'Gene',
 'Allele', 'Annotation', 'Ref_allele', 'Alt_allele','label'])]

,Position,Gene,Allele,Ref_allele,Alt_allele,Annotation,label,accumulated_gain


##### Read Allele count and generation data from metasheet

In [7]:
df_Alt_allele_count_all_generation = pd.read_excel("../../../data/MetaData_ecoli_final.xlsx", sheet_name= "Mastersheet")
df_Alt_allele_count_all_generation.head()

,CHROM,Position,Gene,Allele,Annotation,AC:0,AC:1000,AC:1500,AC:2000,AC:2500,...,AC:57000,AC:57500,AC:58000,AC:58500,AC:59000,AC:59500,AC:60000,Ref_allele,Alt_allele,label
0,chrI,1554,thrA,C->A,missense,0,1,0,2,1.0,...,0,0,1.0,0.0,1.0,0.0,0.0,C,A,p1
1,chrI,1556,thrA,C->A,synonymous,0,0,0,2,0.0,...,0,0,0.0,0.0,0.0,0.0,0.0,C,A,p1
2,chrI,2450,thrA,G->T,synonymous,0,0,0,0,0.0,...,81,41,109.0,129.0,4.0,67.0,8.0,G,T,p1
3,chrI,4274,thrC,C->T,missense,0,0,0,0,0.0,...,0,0,0.0,0.0,0.0,0.0,0.0,C,T,p1
4,chrI,6400,yaaA,T->G,missense,0,0,0,0,2.0,...,0,0,0.0,0.0,0.0,0.0,0.0,T,G,p1


##### Check if any duplicated row

In [8]:
df_Alt_allele_count_all_generation[df_Alt_allele_count_all_generation.duplicated(subset=['Position', 'Gene', 'Allele', 'Annotation', 'Ref_allele', 'Alt_allele','label'])]

,CHROM,Position,Gene,Allele,Annotation,AC:0,AC:1000,AC:1500,AC:2000,AC:2500,...,AC:57000,AC:57500,AC:58000,AC:58500,AC:59000,AC:59500,AC:60000,Ref_allele,Alt_allele,label
2028,chrI,450000,ylaB,A->G,missense,0,0,0,0,0.0,...,0,0,0.0,0.0,0.0,0.0,0.0,A,G,p3


##### delete duplicate row in df_Alt_allele_count_all_generation dataframe where duplicate indicate when ['CHROM','Position', 'Gene', 'Allele', 'Annotation', 'Ref_allele', 'Alt_allele','label'] are same in mutiple row


In [9]:
df_Alt_allele_count_all_generation.drop_duplicates(subset=['CHROM','Position', 'Gene', 'Allele', 'Annotation', 'Ref_allele', 'Alt_allele','label'], inplace=True)
df_Alt_allele_count_all_generation.reset_index(inplace=True, drop=True)
# df_Alt_allele_count_all_generation

In [10]:
df_Alt_allele_count_all_generation.head()

,CHROM,Position,Gene,Allele,Annotation,AC:0,AC:1000,AC:1500,AC:2000,AC:2500,...,AC:57000,AC:57500,AC:58000,AC:58500,AC:59000,AC:59500,AC:60000,Ref_allele,Alt_allele,label
0,chrI,1554,thrA,C->A,missense,0,1,0,2,1.0,...,0,0,1.0,0.0,1.0,0.0,0.0,C,A,p1
1,chrI,1556,thrA,C->A,synonymous,0,0,0,2,0.0,...,0,0,0.0,0.0,0.0,0.0,0.0,C,A,p1
2,chrI,2450,thrA,G->T,synonymous,0,0,0,0,0.0,...,81,41,109.0,129.0,4.0,67.0,8.0,G,T,p1
3,chrI,4274,thrC,C->T,missense,0,0,0,0,0.0,...,0,0,0.0,0.0,0.0,0.0,0.0,C,T,p1
4,chrI,6400,yaaA,T->G,missense,0,0,0,0,2.0,...,0,0,0.0,0.0,0.0,0.0,0.0,T,G,p1


##### Read LLR Score

In [11]:
df_esm = pd.read_excel("../../../data/LLR.xlsx")
df_esm.head()

,Position,Gene,Allele,Ref_allele,Alt_allele,Annotation,label,Start,End,Strand,...,has_space,WT_aa_made2,comparison2,Mut_aa_made,SNP,seq_id_x,universal_id,seq_id_y,mut_name,esm_score
0,241,thrL,A->C,A,C,missense,m1,190,255,+,...,False,MKRISTTITTTITITTGNGAG,True,MKRISTTITTTITITTGHGAG,N18H,241_thrL,241_thrL_N18H,241_thrL,N18H,-4.279933
1,1615,thrA,T->G,T,G,missense,m1,336,2798,+,...,False,MRVLKFGGTSVANAERFLRVADILESNARQGQVATVLSAPAKITNH...,True,MRVLKFGGTSVANAERFLRVADILESNARQGQVATVLSAPAKITNH...,I427S,1615_thrA,1615_thrA_I427S,1615_thrA,I427S,-11.530375
2,8430,talB,T->G,T,G,missense,m1,8236,9189,+,...,False,MTDKLTSLRQYTTVVADTGDIAAMKLYQPQDATTNPSLILNAAQIP...,True,MTDKLTSLRQYTTVVADTGDIAAMKLYQPQDATTNPSLILNAAQIP...,D65E,8430_talB,8430_talB_D65E,8430_talB,D65E,-5.331029
3,12527,dnaK,A->C,A,C,missense,m1,12161,14077,+,...,False,MGKIIGIDLGTTNSCVAIMDGTTPRVLENAEGDRTTPSIIAYTQDG...,True,MGKIIGIDLGTTNSCVAIMDGTTPRVLENAEGDRTTPSIIAYTQDG...,M123L,12527_dnaK,12527_dnaK_M123L,12527_dnaK,M123L,-4.669276
4,13974,dnaK,A->C,A,C,missense,m1,12161,14077,+,...,False,MGKIIGIDLGTTNSCVAIMDGTTPRVLENAEGDRTTPSIIAYTQDG...,True,MGKIIGIDLGTTNSCVAIMDGTTPRVLENAEGDRTTPSIIAYTQDG...,Q605P,13974_dnaK,13974_dnaK_Q605P,13974_dnaK,Q605P,-7.992450


##### Drop Duplicates if any in df_esm

In [12]:
df_esm_no_duplicates = df_esm[["Position",  "Gene", "label", "esm_score", "Allele"]].drop_duplicates(subset= ["Position",  "Gene", "label", "Allele"], keep='first')
df_esm_no_duplicates.dropna(inplace=True)
df_esm_no_duplicates.reset_index(inplace=True, drop=True)
df_esm_no_duplicates.head()

,Position,Gene,label,esm_score,Allele
0,241,thrL,m1,-4.279933,A->C
1,1615,thrA,m1,-11.530375,T->G
2,8430,talB,m1,-5.331029,T->G
3,12527,dnaK,m1,-4.669276,A->C
4,13974,dnaK,m1,-7.992450,A->C


##### Merge Gain score and allele count

In [13]:
df_complete = pd.merge(df_Gain_calculated,df_Alt_allele_count_all_generation, on=['Position','Gene','Allele','Annotation','Ref_allele','Alt_allele','label'], how = "left")
df_complete.head()

,Position,Gene,Allele,Ref_allele,Alt_allele,Annotation,label,accumulated_gain,CHROM,AC:0,...,AC:55500,AC:56000,AC:56500,AC:57000,AC:57500,AC:58000,AC:58500,AC:59000,AC:59500,AC:60000
0,63,intergenic,A->C,A,C,noncoding,p6,-10.385914,chrI,0,...,49,78,45,22,36,39.0,11.0,1.0,1.0,6.0
1,201,thrL,T->G,T,G,missense,p6,-1.750693,chrI,0,...,100,140,93,28,79,62.0,15.0,3.0,0.0,13.0
2,241,thrL,A->C,A,C,missense,m1,-0.911836,chrI,0,...,6,1,2,0,0,0.0,0.0,0.0,0.0,0.0
3,309,thrA,T->G,T,G,noncoding,m1,4.009052,chrI,0,...,0,0,0,0,0,0.0,1.0,2.0,7.0,9.0
4,322,thrA,A->G,A,G,noncoding,p3,1.532477,chrI,0,...,23,24,13,16,26,8.0,30.0,22.0,23.0,0.0


##### Extract the column relavent for current analysis from allele count dataframe (all the generation wise allele count, 'CHROM','Position', 'Gene', 'Allele', 'Annotation', 'Ref_allele', 'Alt_allele','label')


In [14]:
filtered_list = [item for item in df_complete if item.startswith("AC")] ## Fiter the column starting with "AC" which indicate allele count
filtered_list.extend(['CHROM','Position', 'Gene', 'Allele', 'Annotation', 'Ref_allele', 'Alt_allele','label', "accumulated_gain"]) ##Add other relavent information
df_allele_generation = df_complete[filtered_list]
df_allele_generation.head() # Get complete dataframe containing all generation wise allele count and other relavent column

,AC:0,AC:1000,AC:1500,AC:2000,AC:2500,AC:3000,AC:4000,AC:4500,AC:5000,AC:5500,...,AC:60000,CHROM,Position,Gene,Allele,Annotation,Ref_allele,Alt_allele,label,accumulated_gain
0,0,0,0,0,0.0,0,0,0,0,0,...,6.0,chrI,63,intergenic,A->C,noncoding,A,C,p6,-10.385914
1,0,0,0,0,0.0,0,0,0,0,0,...,13.0,chrI,201,thrL,T->G,missense,T,G,p6,-1.750693
2,0,0,0,0,0.0,0,0,0,0,0,...,0.0,chrI,241,thrL,A->C,missense,A,C,m1,-0.911836
3,0,0,0,0,0.0,0,0,0,0,0,...,9.0,chrI,309,thrA,T->G,noncoding,T,G,m1,4.009052
4,0,0,0,0,0.0,0,0,0,0,0,...,0.0,chrI,322,thrA,A->G,noncoding,A,G,p3,1.532477


##### Define mutator and non-mutator population 

In [15]:
mutator_list = ['m1', 'm2', 'm3', 'm4', 'p3', 'p6']
non_mutator_list = ['p1', 'p2', 'p4', 'p5', 'm5', 'm6']

##### Find the generation which have missing value 

In [16]:
nan_columns = df_allele_generation.columns[df_allele_generation.isna().any()]
df_with_nan = df_allele_generation[nan_columns]
nan_counts = df_allele_generation.isna().sum()
nan_counts = nan_counts[nan_counts > 0]
print("\n Number of NaN values in each column:")
print(nan_counts)


 Number of NaN values in each column:
AC:2500      355
AC:15000    4953
AC:46500     331
AC:58000    3753
AC:58500     355
AC:59000     355
AC:59500     355
AC:60000     355
dtype: int64


##### Drop the generation having nan values

In [17]:
df_allele_generation.drop(nan_counts.index.tolist(), axis = 1, inplace=True)
df_allele_generation.reset_index(inplace=True, drop = True)
df_allele_generation.head()

,AC:0,AC:1000,AC:1500,AC:2000,AC:3000,AC:4000,AC:4500,AC:5000,AC:5500,AC:6000,...,AC:57500,CHROM,Position,Gene,Allele,Annotation,Ref_allele,Alt_allele,label,accumulated_gain
0,0,0,0,0,0,0,0,0,0,0,...,36,chrI,63,intergenic,A->C,noncoding,A,C,p6,-10.385914
1,0,0,0,0,0,0,0,0,0,0,...,79,chrI,201,thrL,T->G,missense,T,G,p6,-1.750693
2,0,0,0,0,0,0,0,0,0,0,...,0,chrI,241,thrL,A->C,missense,A,C,m1,-0.911836
3,0,0,0,0,0,0,0,0,0,0,...,0,chrI,309,thrA,T->G,noncoding,T,G,m1,4.009052
4,0,0,0,0,0,0,0,0,0,0,...,26,chrI,322,thrA,A->G,noncoding,A,G,p3,1.532477


##### Create dataset having all mutation info in proper form for easy calculation
##### Make a dataframe having column ['label', 'CHROM', 'Position', 'Gene', 'Allele', 'Annotation', 'generation', 'allele count', 'generation_number']

In [18]:
data_all_copy = df_allele_generation.copy()
all_mutation = pd.DataFrame() # The all_mutation dataframe contain generation and population wise accumulated gain

for i in data_all_copy.columns[:-9]:
    new_df_tmp = pd.DataFrame()
    new_df = data_all_copy[data_all_copy.loc[:, i]!=0]
    new_df_tmp = new_df[["label", 'CHROM', 'Position', 'Gene', 'Allele', 'Annotation']]
    new_df_tmp.loc[:, "generation"] = i
    new_df_tmp.loc[:, "AG"] = new_df.loc[:, "accumulated_gain"]
    new_df_tmp.loc[:, "allele count"] = new_df.loc[:, i]
    all_mutation = pd.concat([all_mutation, new_df_tmp], axis = 0)
    
    
all_mutation.reset_index(inplace=True, drop=True)
all_mutation['generation_number'] = all_mutation['generation'].str.extract(r'(\d+)').astype(int)
for i in range(all_mutation.shape[0]):
    
    if all_mutation.loc[i, "label"] in mutator_list:
        all_mutation.loc[i, "mutator"] = "mutator"
    if all_mutation.loc[i, "label"] in non_mutator_list:
        all_mutation.loc[i, "mutator"] = "non mutator"

all_mutation.head()

,label,CHROM,Position,Gene,Allele,Annotation,generation,AG,allele count,generation_number,mutator
0,m6,chrI,1457,thrA,G->A,missense,AC:0,-8.160701,1,0,non mutator
1,m2,chrI,2098,thrA,A->G,missense,AC:0,0.007780,1,0,mutator
2,m2,chrI,2942,thrB,A->G,missense,AC:0,-4.953477,1,0,mutator
3,m4,chrI,5437,yaaX,A->G,missense,AC:0,-10.279634,1,0,mutator
4,m5,chrI,7129,yaaJ,A->C,missense,AC:0,2.182288,2,0,non mutator


##### Remove white spaces from columns

In [19]:
# df_esm_no_duplicates["Gene"] = df_esm_no_duplicates["Gene"].str.strip()
all_mutation["Gene"] = all_mutation["Gene"].str.strip()
# df_esm_no_duplicates["Allele"] = df_esm_no_duplicates["Allele"].str.strip()
all_mutation["Allele"] = all_mutation["Allele"].str.strip()
all_mutation["Allele"] = all_mutation["Allele"].str.strip()
df_Gain_calculated["Allele"] = df_Gain_calculated["Allele"].str.strip()

In [20]:
all_mutation["Gene"] = all_mutation["Gene"].str.strip()
df_Gain_calculated["Gene"] = df_Gain_calculated["Gene"].str.strip()

In [21]:
all_mutation

,label,CHROM,Position,Gene,Allele,Annotation,generation,AG,allele count,generation_number,mutator
0,m6,chrI,1457,thrA,G->A,missense,AC:0,-8.160701,1,0,non mutator
1,m2,chrI,2098,thrA,A->G,missense,AC:0,0.007780,1,0,mutator
2,m2,chrI,2942,thrB,A->G,missense,AC:0,-4.953477,1,0,mutator
3,m4,chrI,5437,yaaX,A->G,missense,AC:0,-10.279634,1,0,mutator
4,m5,chrI,7129,yaaJ,A->C,missense,AC:0,2.182288,2,0,non mutator
...,...,...,...,...,...,...,...,...,...,...,...
1048642,m3,chrI,4629225,lasT,C->T,missense,AC:57500,-5.614587,72,57500,mutator
1048643,p6,chrI,4629227,lasT,A->C,synonymous,AC:57500,-5.942512,1,57500,mutator
1048644,m2,chrI,4629386,lasT,A->G,synonymous,AC:57500,7.409069,1,57500,mutator
1048645,m2,chrI,4629441,lasT,C->T,missense,AC:57500,-8.417367,4,57500,mutator


In [22]:
# all_mutation.rename(columns = {"accumulated_gain":"kGain 10 mer"}, inplace=True)


##### Extract all mutation of 57500 generation

In [23]:
df_gen_57500 = all_mutation[['label', 'CHROM', 'Position', 'Gene', 'Allele', "generation_number"]]
df_gen_57500 = df_gen_57500[df_gen_57500["generation_number"]==57500]
df_gen_57500 = pd.merge(df_gen_57500[[ 'Gene', 'Position', "label", "Allele", "generation_number"]], df_Gain_calculated, on = [ 'Gene', 'Position', "label", "Allele"])
df_gen_57500

,Gene,Position,label,Allele,generation_number,Ref_allele,Alt_allele,Annotation,accumulated_gain
0,intergenic,63,p6,A->C,57500,A,C,noncoding,-10.385914
1,thrL,201,p6,T->G,57500,T,G,missense,-1.750693
2,thrA,322,p3,A->G,57500,A,G,noncoding,1.532477
3,thrA,535,m2,C->T,57500,C,T,missense,-5.822150
4,thrA,1170,p3,C->T,57500,C,T,missense,2.256065
...,...,...,...,...,...,...,...,...,...
15255,lasT,4629225,m3,C->T,57500,C,T,missense,-5.614587
15256,lasT,4629227,p6,A->C,57500,A,C,synonymous,-5.942512
15257,lasT,4629386,m2,A->G,57500,A,G,synonymous,7.409069
15258,lasT,4629441,m2,C->T,57500,C,T,missense,-8.417367


In [24]:
def return_mutated_fasta(data, fasta_seq, generation_number):

    data = data.copy()

    tmp = data[['label', 'CHROM', 'Position', 'Gene', 'Allele', "generation_number"]].drop_duplicates()
    tmp_df_new = tmp[tmp["generation_number"]==generation_number]
    tmp_df_new = pd.merge(tmp_df_new[[ 'Gene', 'Position', "label", "Allele", "generation_number"]], df_Gain_calculated, on = [ 'Gene', 'Position', "label", "Allele"])
    # df_gen_57500

    dct_fasta_sequence = dict()

    def replace_multiple_chars(s, positions, new_chars):
        s_list = list(s)  # Convert to mutable list
        
        for pos, char in zip(positions, new_chars):
            if 0 <= pos < len(s_list):  # Ensure position is within bounds
                s_list[pos] = char
        
        return ''.join(s_list)  # Convert back to string


    for i in set(tmp_df_new["label"]):

        tmp_df = tmp_df_new[tmp_df_new["label"]==i]

        tmp_df.reset_index(inplace=True, drop=True)

        dct_fasta_sequence[i] = fasta_seq

        for j in range(tmp_df.shape[0]):

            dct_fasta_sequence[i] = replace_multiple_chars(dct_fasta_sequence[i], positions = list(tmp_df["Position"].astype(int)-1), new_chars = list(tmp_df["Alt_allele"]))

        # break

    return dct_fasta_sequence



##### Store mutated fasta population wise in a dictionary

In [25]:
dct_fasta_57500 = return_mutated_fasta(data = all_mutation, fasta_seq = seq_fasta, generation_number = 57500)
dct_fasta_0 = return_mutated_fasta(data = all_mutation, fasta_seq = seq_fasta, generation_number = 0)

In [26]:
all_mutation

,label,CHROM,Position,Gene,Allele,Annotation,generation,AG,allele count,generation_number,mutator
0,m6,chrI,1457,thrA,G->A,missense,AC:0,-8.160701,1,0,non mutator
1,m2,chrI,2098,thrA,A->G,missense,AC:0,0.007780,1,0,mutator
2,m2,chrI,2942,thrB,A->G,missense,AC:0,-4.953477,1,0,mutator
3,m4,chrI,5437,yaaX,A->G,missense,AC:0,-10.279634,1,0,mutator
4,m5,chrI,7129,yaaJ,A->C,missense,AC:0,2.182288,2,0,non mutator
...,...,...,...,...,...,...,...,...,...,...,...
1048642,m3,chrI,4629225,lasT,C->T,missense,AC:57500,-5.614587,72,57500,mutator
1048643,p6,chrI,4629227,lasT,A->C,synonymous,AC:57500,-5.942512,1,57500,mutator
1048644,m2,chrI,4629386,lasT,A->G,synonymous,AC:57500,7.409069,1,57500,mutator
1048645,m2,chrI,4629441,lasT,C->T,missense,AC:57500,-8.417367,4,57500,mutator


##### Verify few position

In [27]:
pos = 77883
seq_fasta[pos-1], dct_fasta_57500["m5"][pos-1]

('C', 'T')

In [28]:
m1_population = pd.read_excel("../../../data/m1_annotated_timecourse.xlsx","Sheet2") 
m1_population["label"] = "m1" 
m1_population = m1_population[['Position','Gene','Allele','Annotation','Test.statistic','P-value','Ref_allele','Alt_allele','ref_flanking_regions','alt_flanking_regions','label']]

m2_population = pd.read_excel("../../../data/m2_annotated_timecourse.xlsx","Sheet2") 
m2_population["label"] = "m2" 
m2_population = m2_population[['Position','Gene','Allele','Annotation','Test.statistic','P-value','Ref_allele','Alt_allele','ref_flanking_regions','alt_flanking_regions','label']]

m3_population = pd.read_excel("../../../data/m3_annotated_timecourse.xlsx","Sheet2") 
m3_population["label"] = "m3" 
m3_population = m3_population[['Position','Gene','Allele','Annotation','Test.statistic','P-value','Ref_allele','Alt_allele','ref_flanking_regions','alt_flanking_regions','label']]

m4_population = pd.read_excel("../../../data/m4_annotated_timecourse.xlsx","Sheet2") 
m4_population["label"] = "m4" 
m4_population = m4_population[['Position','Gene','Allele','Annotation','Test.statistic','P-value','Ref_allele','Alt_allele','ref_flanking_regions','alt_flanking_regions','label']]

m5_population = pd.read_excel("../../../data/m5_annotated_timecourse.xlsx","Sheet2") 
m5_population["label"] = "m5" 
m5_population = m5_population[['Position','Gene','Allele','Annotation','Test.statistic','P-value','Ref_allele','Alt_allele','ref_flanking_regions','alt_flanking_regions','label']]

m6_population = pd.read_excel("../../../data/m6_annotated_timecourse.xlsx","Sheet2") 
m6_population["label"] = "m6" 
m6_population = m6_population[['Position','Gene','Allele','Annotation','Test.statistic','P-value','Ref_allele','Alt_allele','ref_flanking_regions','alt_flanking_regions','label']]

p1_population = pd.read_excel("../../../data/p1_annotated_timecourse.xlsx","Sheet2") 
p1_population["label"] = "p1" 
p1_population = p1_population[['Position','Gene','Allele','Annotation','Test.statistic','P-value','Ref_allele','Alt_allele','ref_flanking_regions','alt_flanking_regions','label']]

p2_population = pd.read_excel("../../../data/p2_annotated_timecourse.xlsx","Sheet2") 
p2_population["label"] = "p2" 
p2_population = p2_population[['Position','Gene','Allele','Annotation','Test.statistic','P-value','Ref_allele','Alt_allele','ref_flanking_regions','alt_flanking_regions','label']]

p3_population = pd.read_excel("../../../data/p3_annotated_timecourse.xlsx","Sheet2") 
p3_population["label"] = "p3" 
p3_population = p3_population[['Position','Gene','Allele','Annotation','Test.statistic','P-value','Ref_allele','Alt_allele','ref_flanking_regions','alt_flanking_regions','label']]

p4_population = pd.read_excel("../../../data/p4_annotated_timecourse.xlsx","Sheet2") 
p4_population["label"] = "p4" 
p4_population = p4_population[['Position','Gene','Allele','Annotation','Test.statistic','P-value','Ref_allele','Alt_allele','ref_flanking_regions','alt_flanking_regions','label']]

p5_population = pd.read_excel("../../../data/p5_annotated_timecourse.xlsx","Sheet2") 
p5_population["label"] = "p5" 
p5_population = p5_population[['Position','Gene','Allele','Annotation','Test.statistic','P-value','Ref_allele','Alt_allele','ref_flanking_regions','alt_flanking_regions','label']]

p6_population = pd.read_excel("../../../data/p6_annotated_timecourse.xlsx","Sheet2") 
p6_population["label"] = "p6" 
p6_population = p6_population[['Position','Gene','Allele','Annotation','Test.statistic','P-value','Ref_allele','Alt_allele','ref_flanking_regions','alt_flanking_regions','label']]

df = pd.concat([m1_population,m2_population,m3_population,m4_population,m5_population,m6_population,p1_population,p2_population,p3_population,p4_population,p5_population,p6_population])
df.reset_index(inplace=True, drop=True)

fcgr_dict = dict()

for i in dct_fasta_57500.keys():

    fcgr_dict[i] = kaos.chaos_frequency_dictionary(fasta_string= dct_fasta_57500[i], kmer_length=kmer_length, pseudo_count=True, chaos_game_kmer_array=kaos.chaos_game_representation_key(kmer_length = kmer_length))

df["Gene"] = df["Gene"].str.strip()
df["Allele"] = df["Allele"].str.strip()

df_sliding = pd.DataFrame(columns = ['Position', 'Gene', 'Allele', 'Annotation', 'Test.statistic', 'P-value',
       'Ref_allele', 'Alt_allele', 'ref_flanking_regions',
       'alt_flanking_regions', 'label'])
df_complete = pd.DataFrame(columns = ['Position', 'Gene', 'Allele', 'Annotation', 'Test.statistic', 'P-value',
       'Ref_allele', 'Alt_allele', 'ref_flanking_regions',
       'alt_flanking_regions', 'label'])

for i in range(df.shape[0]):

    pop = df.loc[i, "label"]
    
    ref_seq = df.loc[i, "ref_flanking_regions"]
    alt_seq = df.loc[i, "alt_flanking_regions"]
    
    center_pos = int(len(ref_seq)/2)
    
    gain = 0
    
    for j in range(kmer_length):
        
        df_sliding.loc[j, "altered sliding window"] = alt_seq[center_pos - kmer_length+j+1:center_pos+1+j]
        df_sliding.loc[j, "reference sliding window"] = ref_seq[center_pos - kmer_length+j+1:center_pos+j+1]

        df_sliding.loc[j, "gain"] = np.log(fcgr_dict[pop][df_sliding.loc[j, "altered sliding window"]]/ fcgr_dict[pop][df_sliding.loc[j, "reference sliding window"]])
    
        gain = gain + df_sliding.loc[j, "gain"]
    
    df_sliding['Position'] = df.loc[i, 'Position']
    df_sliding['Gene'] = df.loc[i, 'Gene']
    df_sliding['Allele'] = df.loc[i, 'Allele']
    df_sliding['Annotation'] = df.loc[i, 'Annotation']
    df_sliding['Test.statistic'] = df.loc[i, 'Test.statistic']
    df_sliding['P-value'] = df.loc[i, 'P-value']
    df_sliding['Ref_allele'] = df.loc[i, 'Ref_allele']
    df_sliding['Alt_allele'] = df.loc[i, 'Alt_allele']
    df_sliding['ref_flanking_regions'] = df.loc[i, 'ref_flanking_regions']
    df_sliding['alt_flanking_regions'] = df.loc[i, 'alt_flanking_regions']
    df_sliding['label'] = df.loc[i, 'label']
    df_sliding["accumulated_gain"] = gain

    df_complete = pd.concat([df_complete, df_sliding])


df_complete_groupby = df_complete.groupby(["Position", "Gene", "Allele", "Ref_allele", "Alt_allele", "Annotation", "label"])["accumulated_gain"].mean().reset_index()
df_complete_groupby


/tmp/ipykernel_1155840/1595551989.py:101: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_complete = pd.concat([df_complete, df_sliding])


,Position,Gene,Allele,Ref_allele,Alt_allele,Annotation,label,accumulated_gain
0,63,intergenic,A->C,A,C,noncoding,p6,-1.937942
1,201,thrL,T->G,T,G,missense,p6,1.157555
2,241,thrL,A->C,A,C,missense,m1,-0.911836
3,309,thrA,T->G,T,G,noncoding,m1,3.989634
4,322,thrA,A->G,A,G,noncoding,p3,8.106801
...,...,...,...,...,...,...,...,...
36917,4629541,lasT,C->T,C,T,missense,m2,-5.104473
36918,4629573,lasT,T->C,T,C,synonymous,m4,-7.698813
36919,4629600,lasT,A->G,A,G,missense,p3,5.607030
36920,4629600,lasT,A->G,A,G,missense,p6,2.753987


In [29]:
# df_complete_groupby.rename({"accumulated_gain": "kGain 57.5k"}, inplace=True)
# df_complete_groupby.to_csv("accumulated_gain with 57.5k as reference.csv", index=False)

##### pickle population wise mutated fasta wrt 57.5k generation

In [30]:
# import pickle



# # Open a file in write-binary mode and save the data
# with open("dct_fasta_57500.pkl", "wb") as file:
#     pickle.dump(dct_fasta_57500, file)

# print("Data has been pickled successfully!")
